In [ ]:
from pyrogram import Client
from datetime import datetime
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

api_id = os.getenv('api_id')
api_hash = os.getenv('api_hash')

In [ ]:
app = Client("my_account", api_id, api_hash)

# chat_id é o interno do telegram para o chat
# chat_id=-1001746519249
chat_id = -1001251759593  # Allan Santos  # Grupo telegram de notícias

In [ ]:
def convert_date(unix_time):
    ts = int(unix_time)
    # if you encounter a "year is out of range" error the timestamp
    # may be in milliseconds, try `ts /= 1000` in that case
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')


async def list_dialogs():
    async for dialog in app.iter_dialogs():
        print(dialog.chat.first_name or dialog.chat.title, ':', dialog.chat.id)


async def getMessagesFromChat(): # Retorna um objeto Mensagem https://docs.pyrogram.org/api/types/Message#pyrogram.types.Message
    # Ver isso para entender como lidar com midias no telegrma: https://docs.pyrogram.org/api/methods/get_history
    async for m in app.iter_history(chat_id, limit=10, reverse=False):
        print("=============", m.message_id, "==============")
        print('Message id:', m.from_user) # id do usuário
        print('Texto da mensagem:', m.text)
        print('Timestamp: ', convert_date(m.date)) # timestamp da mensagem
        print('É midia?', m.media)
        print('É Audio?', m.audio)
        print('É Foto?', m.photo)
        print('É Legenda', m.caption)
        print('É Sticker?', m.sticker)
        print('É Documento?', m.document)
        print('É Video-note?', m.video_note)
        print('Infos da Web-page?', m.web_page)
        print('Qtd de view do chat?', m.views)
        print('Link da mensagem dentro do grupo:', m.link)
        print('É mensagem automática (telegram)?', m.service) # service (str, optional) – The message is a service message. This field will contain the name of the service message. A service message has one and only one of these fields set: new_chat_members, left_chat_member, new_chat_title, new_chat_photo, delete_chat_photo, group_chat_created, channel_chat_created, migrate_to_chat_id, migrate_from_chat_id, pinned_message, game_high_score, voice_chat_started, voice_chat_ended, voice_chat_scheduled, voice_chat_members_invited.
        print('Data de edição da mensagem:', convert_date(m.edit_date))
        print("============= ==============")


async def getChatInfoById():
    chat = await app.get_chat(chat_id)
    # print(f'{chat}') # para ver o json completo do objeto Chat
    print('============= / CHAT INFOS / ============= ')
    print('Título: [', chat.title, ']')
    print('Tipo:[', chat.type, ']')
    print('============= / FIM / ============= ')

def getChatInfos():
    for dialogs in app.iter_dialogs():
        print(f'{dialogs.chat}')



In [ ]:
with app:
    await app.start()
    await getChatInfoById()
    await app.stop()

In [ ]:
# this will list all telegram chats/groups you participate

with app:
    await app.start()
    await list_dialogs()# 
    await app.stop()

In [ ]:
async def recover_messages(): # Retorna um objeto Mensagem https://docs.pyrogram.org/api/types/Message#pyrogram.types.Message
    m1 = []
    # Ver isso para entender como lidar com midias no telegrma: https://docs.pyrogram.org/api/methods/get_history
    async for m in app.iter_history(chat_id, limit=10, reverse=False):
        m1.append(m) # junta as mensagens num dicionário
        

In [ ]:
with app:
    await app.start()
    await recover_messages()
    await app.stop()

In [ ]:
with app:
    await app.start()
    messages = recover_messages()# 
    await app.stop()


In [ ]:
df = pd.DataFrame()

In [ ]:
# convert to dataframe 
for m in m1:
    x = {
        'message_id' : m.message_id,
        'media_type': m.media,
        'text': m.text,
        'subtitle_picture': m.caption,
        'date_message_sent' : convert_date(m.date),
        'is_audio': m.audio,
        'is_photo': m.photo,
        'is_sticker': m.sticker,
        'is_document': m.document,
        'is_video_note': m.video_note,
        'is_web-page': m.web_page,
        'message_views': m.views,
        'telegram_message_url': m.link, 
        'is_telegram_internal_message': m.service, # service (str, optional) – The message is a service message. This field will contain the name of the service message. A service message has one and only one of these fields set: new_chat_members, left_chat_member, new_chat_title, new_chat_photo, delete_chat_photo, group_chat_created, channel_chat_created, migrate_to_chat_id, migrate_from_chat_id, pinned_message, game_high_score, voice_chat_started, voice_chat_ended, voice_chat_scheduled, voice_chat_members_invited.
        'date_message_edited': convert_date(m.edit_date)
    
    }
    df = df.append(x, ignore_index = True)
     
    

In [ ]:
df.head()

In [ ]:
# df['is_document'].'df['media_type'] == 'web_page']]

df.loc[df['is_document'].isna()]


In [ ]:
df[df['is_web-page'] != 'N']

In [ ]:
df[df['media_type'].isna()].to_csv('./exports/telegram-sem-filtro.csv')

df.to_csv('./exports/telegram-sem-filtro.csv')